In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time

# chromedriver 路徑
chromedriver_path = "./node_modules/chromedriver/bin/chromedriver"

# 參數
options = Options()
# 無頭模式
options.headless = True

# 設置 WebDriver
service = Service(chromedriver_path)
driver = webdriver.Chrome(service=service, options=options)

# 訪問網頁
PRODUCT = 'F-Y12EB'
_URL = f'https://www.momoshop.com.tw/search/searchShop.jsp?keyword={PRODUCT}'
driver.get(_URL)

# 等待一點時間讓畫面載入
time.sleep(5)
#
base_xpath = "/html/body/div[@id='BodyBase']/div[@class='bt_2_layout searchbox searchListArea selectedtop']/div[@class='searchPrdListArea bookList']/div[@class='listArea']/ul[@class='clearfix']/li[{}]"
# 前 N 個商品
for i in range(1, 11):
    current_base_xpath = base_xpath.format(i)
    # XPath 
    img_xpath = current_base_xpath + "/a[@class='goodsUrl']/div[@class='prdImgWrap prdListSwiper swiper-container manyPics swiper-container-initialized swiper-container-horizontal']/div[@class='swiper-wrapper']/div[@class='swiper-slide swiper-slide-active']/img[@class='prdImg']"
    title_xpath = current_base_xpath + "/a[@class='goodsUrl']/div[@class='prdInfoWrap']/div[@class='prdNameTitle']/h3[@class='prdName']"
    price_xpath = current_base_xpath + "/a[@class='goodsUrl']/div[@class='prdInfoWrap']/p[@class='money']/span[@class='price']/b"

    # 取得資訊
    try:
        product_img = driver.find_element(By.XPATH, img_xpath).get_attribute('src')
        product_title = driver.find_element(By.XPATH, title_xpath).text
        product_price = driver.find_element(By.XPATH, price_xpath).text
    except:
        # 關閉瀏覽器
        driver.quit()
        break
    # 輸出查看
    print(f"商品 {i}")
    print("圖片:", product_img)
    print("標題:", product_title)
    print("價格:", product_price)
    print("-" * 30)

# 關閉瀏覽器
driver.quit()


In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time

# 設定 Selenium 瀏覽器參數
options = Options()
options.add_argument("--headless=new")  # 使用新版 headless 模式
options.add_argument("--disable-gpu")  # 避免 GPU 錯誤
options.add_argument("--no-sandbox")  # 避免 Linux 權限問題
options.add_argument("--disable-blink-features=AutomationControlled")  # 防止 Selenium 被檢測

# 自動安裝 ChromeDriver
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# 設定查詢商品
PRODUCT = "可口可樂"
_URL = f"https://www.momoshop.com.tw/search/searchShop.jsp?keyword={PRODUCT}"

print(f"🔍 正在搜尋商品：{PRODUCT}")
driver.get(_URL)

# 使用 WebDriverWait 等待商品列表加載
try:
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, "listArea"))
    )
    print("✅ 商品列表加載成功！")
except Exception as e:
    print(f"❌ 商品列表加載失敗: {e}")
    driver.quit()
    exit()

# **確認商品數量**
product_elements = driver.find_elements(By.XPATH, "//ul[@class='listAreaUl']/li[@class='listAreaLi']")
num_products = len(product_elements)

if num_products == 0:
    print("⚠️ 沒有找到商品，結束程式。")
    driver.quit()
    exit()
elif num_products == 1:
    print("⚠️ 只有 1 個商品，將只抓取這筆資料。")

# **滾動頁面確保所有商品載入**
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(3)  # 等待 JavaScript 完全載入

# **爬取所有商品**
for i in range(1, min(num_products, 10) + 1):  # 最多抓取 10 個商品
    try:
        base_xpath = f"//ul[@class='listAreaUl']/li[@class='listAreaLi'][{i}]"

        # 產品資訊 XPath
        img_xpath = base_xpath + "//picture/img[@class='prdImg']"
        title_xpath = base_xpath + "//h3[@class='prdName']"
        price_xpath = base_xpath + "//span[@class='price']/b"

        # 取得商品圖片
        product_img = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.XPATH, img_xpath))
        ).get_attribute("src")

        # 取得商品標題
        product_title = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.XPATH, title_xpath))
        ).text

        # 取得商品價格
        product_price = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.XPATH, price_xpath))
        ).text

        # **顯示結果**
        print(f"📌 商品 {i}")
        print("📷 圖片：", product_img)
        print("🏷 標題：", product_title)
        print("💰 價格：", product_price)
        print("-" * 50)

    except Exception as e:
        print(f"⚠️ 商品 {i} 無法獲取：{e}")
        continue

# **關閉瀏覽器**
driver.quit()
print("🚪 瀏覽器已關閉")


🔍 正在搜尋商品：可口可樂
✅ 商品列表加載成功！
📌 商品 1
📷 圖片： https://i3.momoshop.com.tw/1727845514/goodsimg/0010/076/881/10076881_OR_m.webp
🏷 標題： 【Coca-Cola 可口可樂】易開罐330ml x24入/箱
💰 價格： 315
--------------------------------------------------
📌 商品 2
📷 圖片： https://i3.momoshop.com.tw/1693991626/goodsimg/0005/327/666/5327666_OR_m.webp
🏷 標題： 【Coca-Cola 可口可樂】易開罐250ml x24入/箱
💰 價格： 259
--------------------------------------------------
📌 商品 3
📷 圖片： https://i2.momoshop.com.tw/1727845423/goodsimg/0005/327/725/5327725_OR_m.webp
🏷 標題： 【Coca-Cola 可口可樂】寶特瓶600ml x24入/箱
💰 價格： 539
--------------------------------------------------
📌 商品 4
📷 圖片： https://i1.momoshop.com.tw/1738825217/goodsimg/0010/046/012/10046012_OR_m.webp
🏷 標題： 【Coca-Cola 可口可樂】纖維+ 寶特瓶600ml x2箱(共48入;24入/箱)
💰 價格： 1,118
--------------------------------------------------
📌 商品 5
📷 圖片： https://i1.momoshop.com.tw/1692399565/goodsimg/0010/696/600/10696600_OR_m.webp
🏷 標題： 【Coca-Cola 可口可樂】易開罐250ml x3箱(共72入;24入/箱)
💰 價格： 687
------------------------------------------------